<a href="https://colab.research.google.com/github/PatWoo23/CreditRiskAnalysis_Project/blob/main/XGBoostModel/2_ProcessingforML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy

# Machine Learning related libraries
# For lable encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
# Importing libraries for evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
# Importing libraries for graphs
from sklearn.metrics import precision_recall_curve, roc_curve

# Importing Logistic Regression, and Random Forest from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# Importing XGBoost
from xgboost import XGBClassifier

# Importing libraries for SHAP
import shap

In [ ]:
# Import excel. First column as index. First row as header.
df0 = pd.read_excel("/content/drive/MyDrive/MIS561/1208_ML0.xlsx", index_col=0, header=0)
pd.set_option('display.max_columns', None)
# Make a deep copy of df0 to df
df = df0.copy(deep=True)
print(df.shape)
print(df.columns)

(36116, 19)
Index(['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NUM_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'Age', 'NUM_FAMILY_MEMBERS',
       'Employment Years', 'Average Status in Initial Phase',
       'Average Status in Current Phase', 'Risky Customers',
       'Client Classification', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL'],
      dtype='object')


In [ ]:
print(df.head())

         CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  NUM_CHILDREN  \
ID                                                                  
5026168            1             1                1             0   
5035920            0             1                1             1   
5041710            1             1                1             3   
5045517            1             1                0             1   
5095015            0             0                1             0   

         AMT_INCOME_TOTAL      NAME_INCOME_TYPE  \
ID                                                
5026168          225000.0             Pensioner   
5035920          180000.0  Commercial associate   
5041710          180000.0               Working   
5045517          180000.0               Working   
5095015          225000.0  Commercial associate   

                   NAME_EDUCATION_TYPE NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  \
ID                                                                            

In [ ]:
# Display the unqiue values of categorical variables
unique_values = {
    'NAME_INCOME_TYPE': df['NAME_INCOME_TYPE'].unique(),
    'NAME_EDUCATION_TYPE': df['NAME_EDUCATION_TYPE'].unique(),
    'NAME_FAMILY_STATUS': df['NAME_FAMILY_STATUS'].unique(),
    'NAME_HOUSING_TYPE': df['NAME_HOUSING_TYPE'].unique()
}

for column, values in unique_values.items():
    print(f"Unique values in '{column}':")
    print(values)
    print('-' * 50)  # Separator for better readability

Unique values in 'NAME_INCOME_TYPE':
['Pensioner' 'Commercial associate' 'Working' 'State servant' 'Student']
--------------------------------------------------
Unique values in 'NAME_EDUCATION_TYPE':
['Higher education' 'Incomplete higher' 'Secondary / secondary special'
 'Lower secondary' 'Academic degree']
--------------------------------------------------
Unique values in 'NAME_FAMILY_STATUS':
['Married' 'Separated' 'Widow' 'Single / not married']
--------------------------------------------------
Unique values in 'NAME_HOUSING_TYPE':
['House / apartment' 'Rented apartment' 'Municipal apartment'
 'With parents' 'Office apartment' 'Co-op apartment']
--------------------------------------------------


# Transform categorical data into integer.

Do not use get_dummies on GoogleColab because it gives "True" or "False" instead of 1 or 0.

In [ ]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'NAME_INCOME_TYPE' column and save it to 'NEW_INCOME_TYPE'
df['NEW_INCOME_TYPE'] = label_encoder.fit_transform(df['NAME_INCOME_TYPE'])

# Display the mapping of categories to integers
print("Label Mapping for NAME_INCOME_TYPE:")
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print(label_mapping)

print("\nOriginal Column Values and Corresponding Encoded Values:")
print(df[['NAME_INCOME_TYPE', 'NEW_INCOME_TYPE']].drop_duplicates())

Label Mapping for NAME_INCOME_TYPE:
{'Commercial associate': 0, 'Pensioner': 1, 'State servant': 2, 'Student': 3, 'Working': 4}

Original Column Values and Corresponding Encoded Values:
             NAME_INCOME_TYPE  NEW_INCOME_TYPE
ID                                            
5026168             Pensioner                1
5035920  Commercial associate                0
5041710               Working                4
5009210         State servant                2
5010457               Student                3


In [ ]:
# Similarly, transform 'NAME_EDUCATION_TYPE'
label_encoder = LabelEncoder()
df['NEW_EDUCATION_TYPE'] = label_encoder.fit_transform(df['NAME_EDUCATION_TYPE'])

print("Label Mapping for NAME_EDUCATION_TYPE:")
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print(label_mapping)

print("\nOriginal Column Values and Corresponding Encoded Values:")
print(df[['NAME_EDUCATION_TYPE', 'NEW_EDUCATION_TYPE']].drop_duplicates())

Label Mapping for NAME_EDUCATION_TYPE:
{'Academic degree': 0, 'Higher education': 1, 'Incomplete higher': 2, 'Lower secondary': 3, 'Secondary / secondary special': 4}

Original Column Values and Corresponding Encoded Values:
                   NAME_EDUCATION_TYPE  NEW_EDUCATION_TYPE
ID                                                        
5026168               Higher education                   1
5035920              Incomplete higher                   2
5041710  Secondary / secondary special                   4
5025820                Lower secondary                   3
5011033                Academic degree                   0


In [ ]:
# Similarly, transform 'NAME_FAMILY_STATUS'
label_encoder = LabelEncoder()
df['NEW_FAMILY_STATUS'] = label_encoder.fit_transform(df['NAME_FAMILY_STATUS'])

print("Label Mapping for NAME_FAMILY_STATUS:")
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print(label_mapping)

print("\nOriginal Column Values and Corresponding Encoded Values:")
print(df[['NAME_FAMILY_STATUS', 'NEW_FAMILY_STATUS']].drop_duplicates())

Label Mapping for NAME_FAMILY_STATUS:
{'Married': 0, 'Separated': 1, 'Single / not married': 2, 'Widow': 3}

Original Column Values and Corresponding Encoded Values:
           NAME_FAMILY_STATUS  NEW_FAMILY_STATUS
ID                                              
5026168               Married                  0
5132804             Separated                  1
5053152                 Widow                  3
5149960  Single / not married                  2


In [ ]:
# Similarly, transform 'NAME_HOUSING_TYPE'
label_encoder = LabelEncoder()
df['NEW_HOUSING_TYPE'] = label_encoder.fit_transform(df['NAME_HOUSING_TYPE'])

print("Label Mapping for NAME_HOUSING_TYPE:")
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print(label_mapping)

print("\nOriginal Column Values and Corresponding Encoded Values:")
print(df[['NAME_HOUSING_TYPE', 'NEW_HOUSING_TYPE']].drop_duplicates())

Label Mapping for NAME_HOUSING_TYPE:
{'Co-op apartment': 0, 'House / apartment': 1, 'Municipal apartment': 2, 'Office apartment': 3, 'Rented apartment': 4, 'With parents': 5}

Original Column Values and Corresponding Encoded Values:
           NAME_HOUSING_TYPE  NEW_HOUSING_TYPE
ID                                            
5026168    House / apartment                 1
5087482     Rented apartment                 4
5096646  Municipal apartment                 2
5126749         With parents                 5
5010526     Office apartment                 3
5036250      Co-op apartment                 0


In [ ]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 36116 entries, 5026168 to 5150337
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   CODE_GENDER                      36116 non-null  int64  
 1   FLAG_OWN_CAR                     36116 non-null  int64  
 2   FLAG_OWN_REALTY                  36116 non-null  int64  
 3   NUM_CHILDREN                     36116 non-null  int64  
 4   AMT_INCOME_TOTAL                 36116 non-null  float64
 5   NAME_INCOME_TYPE                 36116 non-null  object 
 6   NAME_EDUCATION_TYPE              36116 non-null  object 
 7   NAME_FAMILY_STATUS               36116 non-null  object 
 8   NAME_HOUSING_TYPE                36116 non-null  object 
 9   Age                              36116 non-null  int64  
 10  NUM_FAMILY_MEMBERS               36116 non-null  int64  
 11  Employment Years                 36116 non-null  float64
 12  Average Status 

In [ ]:
# Drop the original columns
df = df.drop(columns=['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE'])
print(df.columns)

Index(['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NUM_CHILDREN',
       'AMT_INCOME_TOTAL', 'Age', 'NUM_FAMILY_MEMBERS', 'Employment Years',
       'Average Status in Initial Phase', 'Average Status in Current Phase',
       'Risky Customers', 'Client Classification', 'FLAG_WORK_PHONE',
       'FLAG_PHONE', 'FLAG_EMAIL', 'NEW_INCOME_TYPE', 'NEW_EDUCATION_TYPE',
       'NEW_FAMILY_STATUS', 'NEW_HOUSING_TYPE'],
      dtype='object')


In [ ]:
# Export df
df.to_excel("/content/drive/MyDrive/MIS561/1208_ML1.xlsx", index=False, header=True)
print("DataFrame successfully exported to /content/drive/MyDrive/MIS561/1208_ML1.xlsx")

DataFrame successfully exported to /content/drive/MyDrive/MIS561/1208_ML1.xlsx


In [ ]:
# For checking
df.to_excel("/content/drive/MyDrive/MIS561/1208_ML1.xlsx", index=False, header=True)
print(df.head())

         CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  NUM_CHILDREN  \
ID                                                                  
5026168            1             1                1             0   
5035920            0             1                1             1   
5041710            1             1                1             3   
5045517            1             1                0             1   
5095015            0             0                1             0   

         AMT_INCOME_TOTAL  Age  NUM_FAMILY_MEMBERS  Employment Years  \
ID                                                                     
5026168          225000.0   66                   2               6.7   
5035920          180000.0   42                   3               1.9   
5041710          180000.0   39                   5               2.3   
5045517          180000.0   32                   3              12.3   
5095015          225000.0   36                   2               1.8   

         Av